# OSSL datasets

> Data loading for the [OSSL dataset](https://explorer.soilspectroscopy.org/)

In [ ]:
#| default_exp datasets.ossl

In [ ]:
#| export
from fastcore.all import *
from pathlib import Path
import pandas as pd
from urllib.request import urlretrieve
import re
import numpy as np

from soilspecdata.types import *

In [ ]:
#| hide
from nbdev.showdoc import show_doc

The official [OSSL documentation](https://soilspectroscopy.github.io/ossl-manual/db-desc.html) provides more details on the dataset and its variables. 

In [ ]:
#| export
class OSSLData:
    "OSSL (Open Soil Spectral Library) data container"
    def __init__(self, 
                 df: pd.DataFrame # dataframe containing OSSL data
                 ):
        self.df = df
        self._parse_columns()        
        self.sample_ids = (df['id.layer_local_c'].values if 'id.layer_local_c' 
                           in df.columns else np.arange(len(df)))

In [ ]:
#| export
@patch
def _parse_columns(self:OSSLData):
    "Parse columns into visnir, mir and properties"
    self.visnir_cols = [c for c in self.df.columns if re.match(r'scan_visnir\.\d+_ref', c)]
    self.mir_cols = [c for c in self.df.columns if re.match(r'scan_mir\.\d+_abs', c)]
    spectral_cols = set(self.visnir_cols + self.mir_cols)
    self.properties_cols = [c for c in self.df.columns if c not in spectral_cols]

In [ ]:
show_doc(OSSLData._parse_columns)

---

[source](https://github.com/franckalbinet/soilspecdata/blob/main/soilspecdata/datasets/ossl.py#L28){target="_blank" style="float:right; font-size:smaller"}

### OSSLData._parse_columns

>      OSSLData._parse_columns ()

*Parse columns into visnir, mir and properties*

In [ ]:
#| export
def get_cache_path(
    dest_dir: str='.soilspecdata', # Name of the cache directory
    ) -> Path: # Path to the cache directory (~/dest_dir)
    "Get cache path for OSSL data"
    return Path.home()/dest_dir

For instance:

In [ ]:
#| eval: false
get_cache_path()

Path('/Users/franckalbinet/.soilspecdata')

The default gzipped file is downloaded from the following URL: [https://storage.googleapis.com/soilspec4gg-public/ossl_all_L0_v1.2.csv.gz](https://storage.googleapis.com/soilspec4gg-public/ossl_all_L0_v1.2.csv.gz)

In [ ]:
#| export
def get_ossl(
    url='https://storage.googleapis.com/soilspec4gg-public/ossl_all_L0_v1.2.csv.gz', # OSSL data gzipped file URL
    force_download=False # if True, force download
    ):
    "Load OSSL data from cache or download it"
    cache_path = get_cache_path()/'ossl_v1.2.csv.gz'
    if not cache_path.exists() or force_download:
        cache_path.parent.mkdir(exist_ok=True)
        urlretrieve(url, cache_path)
        
     # Define date columns
    date_columns = [
        'scan.mir.date.begin_iso.8601_yyyy.mm.dd',
        'scan.mir.date.end_iso.8601_yyyy.mm.dd',
        'scan.visnir.date.begin_iso.8601_yyyy.mm.dd',
        'scan.visnir.date.end_iso.8601_yyyy.mm.dd'
    ]
    
    # Update dtype dictionary without datetime columns
    dtype = {
        # IDs and codes
        'id.layer_local_c': 'string',
        'id.location_olc_txt': 'string',
        'id.dataset.site_ascii_txt': 'string',
        'id.scan_local_c': 'string',
        
        # Categorical text fields
        'layer.texture_usda_txt': 'category',
        'pedon.taxa_usda_txt': 'category',
        'horizon.designation_usda_txt': 'category',
        'location.country_iso.3166_txt': 'category',
        'surveyor.address_utf8_txt': 'category',
        'efferv_usda.a479_class': 'category',
        
        # Text fields
        'scan.mir.model.name_utf8_txt': 'string',
        'scan.mir.model.code_any_txt': 'string',
        'scan.mir.method.optics_any_txt': 'string',
        'scan.mir.method.preparation_any_txt': 'string',
        'scan.mir.license.title_ascii_txt': 'string',
        'scan.mir.license.address_idn_url': 'string',
        'scan.mir.doi_idf_url': 'string',
        'scan.mir.contact.name_utf8_txt': 'string',
        'scan.mir.contact.email_ietf_txt': 'string',
        'scan.visnir.model.name_utf8_txt': 'string',
        'scan.visnir.model.code_any_txt': 'string',
        'scan.visnir.method.optics_any_txt': 'string',
        'scan.visnir.method.preparation_any_txt': 'string',
        'scan.visnir.license.title_ascii_txt': 'string',
        'scan.visnir.license.address_idn_url': 'string',
        'scan.visnir.doi_idf_url': 'string',
        'scan.visnir.contact.name_utf8_txt': 'string',
        'scan.visnir.contact.email_ietf_txt': 'string'
    }
    df = pd.read_csv(cache_path, compression='gzip', dtype=dtype,
                     parse_dates=date_columns)
    return OSSLData(df)

How to use it:

In [ ]:
#| eval: false
ossl = get_ossl(force_download=False)

In [ ]:
#| eval: false
ossl.visnir_cols[:2], ossl.mir_cols[:2], ossl.properties_cols[:2]

(['scan_visnir.350_ref', 'scan_visnir.352_ref'],
 ['scan_mir.600_abs', 'scan_mir.602_abs'],
 ['dataset.code_ascii_txt', 'id.layer_local_c'])

In [ ]:
#| export   
@patch
def _get_valid_spectra_mask(self:OSSLData, 
                            spectra_cols: List[str] # Spectra column names
                            ) -> np.ndarray: # Mask
    "Return mask for samples with all non-null values in spectra"
    return self.df[spectra_cols].notna().all(axis=1)

In [ ]:
show_doc(OSSLData._get_valid_spectra_mask)

---

[source](https://github.com/franckalbinet/soilspecdata/blob/main/soilspecdata/datasets/ossl.py#L98){target="_blank" style="float:right; font-size:smaller"}

### OSSLData._get_valid_spectra_mask

>      OSSLData._get_valid_spectra_mask (spectra_cols:List[str])

*Return mask for samples with all non-null values in spectra*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| spectra_cols | List | Spectra column names |
| **Returns** | **ndarray** | **Mask** |

OSSL gzip archive is formated in a wide format (with metadata, soil properties, visnir and mir spectra as columns). Note that all samples have not been scanned simultaneously with VisNIR and MIR instruments according to the data source/provider. 

As a result, when selecting a subset of columns, e.g. `ossl.mir_cols`, the returned dataframe will have a lot of missing values (`NaN`). The above function return a mask for samples with all non-null values in spectra.

In [ ]:
#| eval: false
ossl.df[ossl.mir_cols]

,scan_mir.600_abs,scan_mir.602_abs,scan_mir.604_abs,scan_mir.606_abs,scan_mir.608_abs,scan_mir.610_abs,scan_mir.612_abs,scan_mir.614_abs,scan_mir.616_abs,scan_mir.618_abs,...,scan_mir.3982_abs,scan_mir.3984_abs,scan_mir.3986_abs,scan_mir.3988_abs,scan_mir.3990_abs,scan_mir.3992_abs,scan_mir.3994_abs,scan_mir.3996_abs,scan_mir.3998_abs,scan_mir.4000_abs
0,1.527853,1.531908,1.532084,1.530892,1.530645,1.531506,1.531582,1.531413,1.532904,1.535459,...,0.356776,0.356642,0.355784,0.354743,0.354104,0.353663,0.353237,0.352923,0.352548,0.352053
1,1.538449,1.543622,1.545751,1.546997,1.549450,1.553714,1.557981,1.561652,1.566082,1.571555,...,0.358399,0.358142,0.357144,0.355980,0.355242,0.354722,0.354217,0.353825,0.353376,0.352798
2,1.619721,1.614226,1.615612,1.620649,1.626406,1.631747,1.636411,1.639527,1.642449,1.646890,...,0.372522,0.372338,0.371425,0.370337,0.369679,0.369245,0.368808,0.368469,0.368084,0.367563
3,1.570129,1.567954,1.573055,1.580834,1.586880,1.590397,1.595117,1.600492,1.603847,1.606447,...,0.357992,0.357734,0.356713,0.355480,0.354681,0.354137,0.353619,0.353217,0.352756,0.352158
4,1.484832,1.484367,1.484977,1.486258,1.488400,1.492040,1.495075,1.496595,1.498354,1.501437,...,0.316249,0.316089,0.315098,0.313910,0.313210,0.312758,0.312312,0.311971,0.311568,0.311044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#| eval: false
mask = ossl.df[ossl.mir_cols].notna().all(axis=1)
print(mask.sum(), 'samples with all non-null values in mir spectra out of the total', len(mask))

85684 samples with all non-null values in mir spectra out of the total 135651


In [ ]:
#| export
@patch
def _extract_wavenumbers(self:OSSLData, 
                         cols: List[str] # column names
                         ):
    "Extract wavenumbers from spectral column names"
    ws = np.array([int(re.search(r'\.(\d+)_', c).group(1)) for c in cols])
    if 'visnir' in cols[0].lower(): 
        return np.array([int(1e7 / w) for w in ws])
    return ws

In [ ]:
show_doc(OSSLData._extract_wavenumbers)

---

[source](https://github.com/franckalbinet/soilspecdata/blob/main/soilspecdata/datasets/ossl.py#L106){target="_blank" style="float:right; font-size:smaller"}

### OSSLData._extract_wavenumbers

>      OSSLData._extract_wavenumbers (cols:List[str])

*Extract wavenumbers from spectral column names*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| cols | List | column names |

For instance, to retrieve the wavenumbers from the MIR columns:

In [ ]:
#| eval: false
ossl._extract_wavenumbers(ossl.mir_cols)

array([ 600,  602,  604, ..., 3996, 3998, 4000], shape=(1701,))

In [ ]:
#| export
@patch
def _extract_measurement_type(self:OSSLData, 
                              cols: List[str] # Spectral column names
                              ) -> str: # `abs` (Absorbance) or `ref` (Reflectance)
    "Extract measurement type from column names"
    types = set(re.search(r'_(\w+)$', c).group(1) for c in cols)
    assert len(types) == 1, f"Mixed measurement types found: {types}"
    return types.pop()

In [ ]:
show_doc(OSSLData._extract_measurement_type)

---

[source](https://github.com/franckalbinet/soilspecdata/blob/main/soilspecdata/datasets/ossl.py#L117){target="_blank" style="float:right; font-size:smaller"}

### OSSLData._extract_measurement_type

>      OSSLData._extract_measurement_type (cols:List[str])

*Extract measurement type from column names*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| cols | List | Spectral column names |
| **Returns** | **str** | **`abs` (Absorbance) or `ref` (Reflectance)** |

For instance, to retrieve the measurement type from the MIR or VISNIR columns:

In [ ]:
#| eval: false
ossl._extract_measurement_type(ossl.visnir_cols), ossl._extract_measurement_type(ossl.mir_cols)

('ref', 'abs')

In [ ]:
#| export
@patch
def _filter_wavelength_range(self:OSSLData, 
                             wavenumbers: np.ndarray, # Wavenumbers
                             spectra: np.ndarray, # Spectra
                             cols: List[str], # Column names
                             wmin: Optional[int]=None, # Min wavenumber
                             wmax: Optional[int]=None # Max wavenumber
                             ) -> Tuple[np.ndarray, np.ndarray, List[str]]: # Filtered wavenumbers, spectra, columns
    "Filter spectra based on wavenumber range"
    if wmin is not None:
        assert wmin >= wavenumbers.min(), f"wmin ({wmin}) must be >= minimum wavenumber ({wavenumbers.min()})"
    if wmax is not None:
        assert wmax <= wavenumbers.max(), f"wmax ({wmax}) must be <= maximum wavenumber ({wavenumbers.max()})"
    if wmin is not None and wmax is not None:
        assert wmin < wmax, f"wmin ({wmin}) must be < wmax ({wmax})"
        
    mask = np.ones(len(wavenumbers), dtype=bool)
    if wmin is not None:
        mask &= wavenumbers >= wmin
    if wmax is not None:
        mask &= wavenumbers <= wmax
    return wavenumbers[mask], spectra[:, mask], [cols[i] for i in np.where(mask)[0]]

In [ ]:
show_doc(OSSLData._filter_wavelength_range)

---

[source](https://github.com/franckalbinet/soilspecdata/blob/main/soilspecdata/datasets/ossl.py#L127){target="_blank" style="float:right; font-size:smaller"}

### OSSLData._filter_wavelength_range

>      OSSLData._filter_wavelength_range (wavenumbers:numpy.ndarray,
>                                         spectra:numpy.ndarray, cols:List[str],
>                                         wmin:Optional[int]=None,
>                                         wmax:Optional[int]=None)

*Filter spectra based on wavenumber range*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| wavenumbers | ndarray |  | Wavenumbers |
| spectra | ndarray |  | Spectra |
| cols | List |  | Column names |
| wmin | Optional | None | Min wavenumber |
| wmax | Optional | None | Max wavenumber |
| **Returns** | **Tuple** |  | **Filtered wavenumbers, spectra, columns** |

In [ ]:
#| eval: false
wavenumbers, spectra, cols = ossl._filter_wavelength_range(
    wavenumbers=ossl._extract_wavenumbers(ossl.visnir_cols), 
    spectra=ossl.df[ossl.visnir_cols].values, 
    cols=ossl.visnir_cols, 
    wmin=4000, wmax=25000
)

print(f'Original wavenumbers: {ossl._extract_wavenumbers(ossl.visnir_cols).min()} - {ossl._extract_wavenumbers(ossl.visnir_cols).max()}')
print(f'Filtered wavenumbers: {wavenumbers.min()} - {wavenumbers.max()}')
print(f'Spectra shape: {spectra.shape}')
print(f'Filtered columns. From: {cols[0]} to: {cols[-1]}')

Original wavenumbers: 4000 - 28571
Filtered wavenumbers: 4000 - 25000
Spectra shape: (135651, 1051)
Filtered columns. From: scan_visnir.400_ref to: scan_visnir.2500_ref


**IMPORTANT**: Not that by default, both VISNIR and MIR spectra are converted to wavenumbers. 

In [ ]:
#| export
@patch 
def get_visnir(self:OSSLData, 
               wmin: Optional[int]=4000, # Min wavenumber
               wmax: Optional[int]=25000, # Max wavenumber
               ) -> SpectraData: # VISNIR data
    "Get VISNIR spectra within specified wavenumber range"
    
    wavenumbers = self._extract_wavenumbers(self.visnir_cols)
    spectra = self.df[self.visnir_cols].values
    wavenumbers, _, filtered_cols = self._filter_wavelength_range(
        wavenumbers, spectra, self.visnir_cols, wmin, wmax
    )
    
    valid_mask = self._get_valid_spectra_mask(filtered_cols)
    df_subset = self.df[valid_mask]
    sample_ids = self.sample_ids[valid_mask]
        
    spectra = df_subset[filtered_cols].values
    measurement_type = self._extract_measurement_type(filtered_cols)
    return SpectraData(
        wavenumbers[::-1], 
        spectra[:, ::-1], 
        measurement_type, 
        sample_ids)

For instance, to retrieve the VISNIR spectra between 8000 and 25000 wavenumbers:

In [ ]:
#| eval: false
visnir_data = ossl.get_visnir(wmin=8000, wmax=25000)
visnir_data.spectra.shape

(64644, 426)

In [ ]:
#| export
@patch 
def get_mir(self:OSSLData, 
            wmin: Optional[int]=600, # Min wavenumber
            wmax: Optional[int]=4000, # Max wavenumber
            ) -> SpectraData: # MIR data
    "Get MIR spectra within specified wavenumber range"
    wavenumbers = self._extract_wavenumbers(self.mir_cols)
    spectra = self.df[self.mir_cols].values
    wavenumbers, _, filtered_cols = self._filter_wavelength_range(
        wavenumbers, spectra, self.mir_cols, wmin, wmax
    )
    
    valid_mask = self._get_valid_spectra_mask(filtered_cols)
    df_subset = self.df[valid_mask]
    sample_ids = self.sample_ids[valid_mask]
        
    spectra = df_subset[filtered_cols].values
    measurement_type = self._extract_measurement_type(filtered_cols)
    
    return SpectraData(wavenumbers, spectra, measurement_type, sample_ids)

For instance, to retrieve the MIR spectra between 600 and 4000 wavenumbers (default range):

In [ ]:
#| eval: false
mir_data = ossl.get_mir()
mir_data.spectra.shape, mir_data.wavenumbers.min(), mir_data.wavenumbers.max()

((85684, 1701), np.int64(600), np.int64(4000))

In [ ]:
#| export
@patch
def get_properties(self:OSSLData, 
                   properties=None, # Properties
                   require_complete: bool=False # if True, only return samples with no null values
                   ) -> pd.DataFrame: # Selected properties data
    "Get properties data with sample IDs"
    if properties is None:
        properties = self.properties_cols
    elif isinstance(properties, str):
        properties = [properties]
            
    df_subset = pd.DataFrame({
        'id': self.sample_ids,
        **{col: self.df[col] for col in properties}
    }).set_index('id')
        
    if require_complete:
        return df_subset.dropna()
    return df_subset

Get only complete MIR spectra:

In [ ]:
#| eval: false
mir_data = ossl.get_mir()

Get properties needed as ML targets (must be complete):

In [ ]:
#| eval: false
targets = ossl.get_properties(['cec_usda.a723_cmolc.kg'], require_complete=True)
targets.shape, targets.head()

((57064, 1),
         cec_usda.a723_cmolc.kg
 id                            
 S40857                6.633217
 S40858                3.822628
 S40859                3.427324
 S40860                1.906545
 S40861               13.403203)

Get optional metadata (can have `NaN` values):

In [ ]:
#| eval: false
metadata = ossl.get_properties(['longitude.point_wgs84_dd', 'latitude.point_wgs84_dd'], require_complete=False)
metadata.shape, metadata.head()

((135651, 2),
            longitude.point_wgs84_dd  latitude.point_wgs84_dd
 id                                                          
 icr072246                 15.687492                -7.377750
 icr072247                 15.687492                -7.377750
 icr072266                 15.687817                -7.351243
 icr072267                 15.687817                -7.351243
 icr072286                 15.687965                -7.331673)

In [ ]:
#| export
@patch
def get_aligned_data(self:OSSLData, 
                    spectra_data: SpectraData, # Spectra data
                    target_cols: Union[str, List[str]] # Target columns
                    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]: # Aligned spectra, targets, sample IDs
    "Get aligned spectra and target data for ML, along with their sample IDs"
    targets = self.get_properties(target_cols, require_complete=True)
    mir_df = pd.DataFrame(spectra_data.spectra, columns=spectra_data.wavenumbers, index=spectra_data.sample_ids)
    common_df = mir_df.join(targets, how='inner')
    features = common_df.iloc[:, :len(spectra_data.wavenumbers)].values
    targets = common_df.iloc[:, len(spectra_data.wavenumbers):].values
    sample_ids = common_df.index.values
    
    return features, targets, sample_ids

For instance, to retrieve the MIR spectra and the corresponding CEC values in an amenable form for a Machine/Deep Learning pipeline:

In [ ]:
#| eval: false
X, y, ids = ossl.get_aligned_data(
    spectra_data=mir_data,
    target_cols='cec_usda.a723_cmolc.kg'
)

X.shape, y.shape, ids.shape

((3, 3), (3, 1), (3,))

Later, if you need metadata for these samples:

In [ ]:
#| eval: false
metadata = ossl.get_properties(['longitude.point_wgs84_dd', 'latitude.point_wgs84_dd']).loc[ids]
metadata.head()

,longitude.point_wgs84_dd,latitude.point_wgs84_dd
id,,
173693,NaN,NaN
172161,-120.354407,42.207350
181527,-107.274835,47.434481
176683,NaN,NaN
212508,-103.316306,46.488522
